In [ ]:
import os

### Paths

In [1]:
# Set the path to the symbol icons directory

model_dir = 'C:/Users/lsreeram/Downloads/_/Hex/AI PID/Model_Dir/03'
train_dir = 'C:/Users/lsreeram/Downloads/_/Hex/AI PID/01 Train_Dataset/'
test_dir = 'C:/Users/lsreeram/Downloads/_/Hex/AI PID/02 Test_Dataset/'
pdf_path = 'C:/Users/lsreeram/Downloads/_/Hex/AI PID/02 Test_Dataset'

In [40]:
poppler_path=r'C:\Program Files\poppler-23.05.0\bin'
pdf_path_ext = 'C:/Users/lsreeram/Downloads/_/Hex/AI PID/02 Test_Dataset/Test.pdf'

In [3]:
import os
import numpy as np
# Perform one-hot encoding on the labels
from sklearn.preprocessing import LabelEncoder

In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [14]:
from pdf2image import convert_from_path

In [ ]:
import cv2
import PyPDF2


In [ ]:
#change_to_test_dir = os.chdir('C:/Users/lsreeram/Downloads/_/Hex/AI PID/02 Test_Dataset/')
#change_to_pdf_path = os.chdir('C:/Users/lsreeram/Downloads/_/Hex/AI PID/02 Test_Dataset')

In [ ]:
#--pip install --upgrade tensorflow --user
#-----pip install pdf2imageinstall
#--pip install opencv-python
#--pip install pdf2image
#--pip install pdf2image opencv-python
##--conda install -c conda-forge poppler
#--pip install PyPDF2

## CNN Model

Training Dataset:
    
Image type			jpeg

Dimensions			64x64

Width x Height		64 pixels

Hori x Vert Resolution  96 dpi

Bit depth			1

In [ ]:
# Get the list of symbol icon files
symbol_icon_files = os.listdir(train_dir)

# Set the input image dimensions
input_shape = (32, 32, 3)

# Initialize lists to store images and labels
images = []
labels = []

In [ ]:
# Load symbol icons and their labels
for symbol_icon_filename in symbol_icon_files:
    label, ext = os.path.splitext(symbol_icon_filename)
    if ext.lower() in ['.jpeg', '.jpg', '.png']:
        label = os.path.splitext(symbol_icon_filename)[0]  # Use the symbol_icon_filename as the label
        ext = os.path.splitext(symbol_icon_filename)[1]
        image_path = os.path.join(train_dir, symbol_icon_filename)
        image = load_img(image_path, target_size=input_shape[:2])
        image_array = img_to_array(image)
        images.append(image_array)
        labels.append(label)

# Convert the data into NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize the image data
images = images / 255.0

In [ ]:
labels

In [ ]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [ ]:
# Create the CNN model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(np.unique(labels_encoded)), activation='softmax')  # Number of classes based on unique labels
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the CNN model
model.fit(images, labels_encoded, epochs=10, batch_size=32)

In [ ]:
os.chdir(model_dir)
# Save the trained model
model.save(model_dir)

## Test Model

In [ ]:
# Function to preprocess the symbol image
def preprocess_image(image):
    # Resize the image to a consistent size
    image = cv2.resize(image, (64, 64))
    # Normalize the pixel values
    image = image / 255.0
    return image

# Function to extract symbols from the PDF image
def extract_symbols_from_pdf(pdf_image):
    # Apply symbol extraction techniques to obtain symbol images
    # This could involve using computer vision techniques like object detection, contour detection, or image segmentation
    symbols = []
    # Extract symbols from pdf_image and add them to the symbols list
    # ...

    return symbols

In [42]:

MAX_IMAGE_SIZE = 1000000  # Maximum image size (in pixels)

def extract_symbols_from_pdf(pdf_path):
    symbols = []
    # Convert PDF pages to images
    images = convert_from_path(pdf_path, dpi=300,poppler_path=poppler_path)
    
    for image in images:
        # Convert PIL image to OpenCV format
        cv2_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        
        # Apply image processing or computer vision techniques to extract symbols
        # Convert the image to grayscale
        gray = cv2.cvtColor(cv2_image, cv2.COLOR_BGR2GRAY)
        
        # Check image size
        if gray.size > MAX_IMAGE_SIZE:
            # Resize the image to a smaller size
            ratio = np.sqrt(MAX_IMAGE_SIZE / gray.size)
            resized_gray = cv2.resize(gray, (0, 0), fx=ratio, fy=ratio)
        else:
            resized_gray = gray
        # Append extracted symbols to the 'symbols' list
        symbols.append(resized_gray)
    
    return symbols


In [41]:
import os
from pdf2image import convert_from_path

pdf_path = 'C:/Users/lsreeram/Downloads/_/Hex/AI PID/02 Test_Dataset/Test.pdf'
def extract_symbols_from_pdf(pdf_path):
    symbols = []
    # Convert PDF pages to images
    images = convert_from_path(pdf_path, poppler_path=poppler_path)

    for image in images:
        # Process the image to extract symbols
        symbols = extract_symbols(image)
        # Perform further processing on the extracted symbols

    return symbols

# Call the function to extract symbols from the PDF
symbols = extract_symbols_from_pdf(pdf_path)


error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [43]:
# Load the complex engineering drawing PDFs and extract symbols
pdf_path = os.chdir('C:/Users/lsreeram/Downloads/_/Hex/AI PID/02 Test_Dataset')
pdf_image = cv2.imread(pdf_path)
symbols = extract_symbols_from_pdf(pdf_image)

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
# Prepare the symbols for prediction
preprocessed_symbols = []
for symbol in symbols:
    preprocessed_symbol = preprocess_image(symbol)
    preprocessed_symbols.append(preprocessed_symbol)

# Convert the symbols list to a NumPy array
symbol_images = np.array(preprocessed_symbols)

In [ ]:
model_dir = os.chdir('C:/Users/lsreeram/Downloads/_/Hex/AI PID/Model_Dir/01')

In [ ]:
# Load the trained model
model = load_model('C:/Users/lsreeram/Downloads/_/Hex/AI PID/Model_Dir/01')

In [ ]:
# Perform predictions on the symbol images
predictions = model.predict(symbol_images)

In [ ]:
# Interpret the predictions
predicted_symbols = []  # List to store the predicted symbols

for prediction in predictions:
    # Assuming prediction is a one-hot encoded vector
    predicted_label = np.argmax(prediction)
    predicted_symbols.append(predicted_label)

# Associate predicted symbols with their corresponding locations in the original drawing
# This step involves integrating the predicted symbols with the appropriate positions or connections in the drawing

# Print the predicted symbols
for symbol, predicted_symbol in zip(symbols, predicted_symbols):
    print("Symbol:", symbol)
    print("Predicted Symbol:", predicted_symbol)


In [ ]:


# Load the trained model
os.chdir('C:/Users/lsreeram/Downloads/_/Hex/AI PID/Model_Dir/02')
model = load_model('saved_model.pb')

# Function to preprocess the symbol image
def preprocess_image(image):
    # Resize the image to a consistent size
    image = cv2.resize(image, (64, 64))
    # Normalize the pixel values
    image = image / 255.0
    return image

# Function to extract symbols from the PDF image
def extract_symbols_from_pdf(pdf_image):
    # Apply symbol extraction techniques to obtain symbol images
    # This could involve using computer vision techniques like object detection, contour detection, or image segmentation
    symbols = []
    # Extract symbols from pdf_image and add them to the symbols list
    # ...

    return symbols


### Continue

In [ ]:

# Function to extract symbols from the PDF
def extract_symbols_from_pdf(pdf_path_ext):
    symbols = []
    # Convert PDF pages to images
    images = convert_from_path(pdf_path_ext, poppler_path=poppler_path)

    # Iterate through the images
    for image in images:
        # Convert image to grayscale
        gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
                # Check image size
        if gray.size > MAX_IMAGE_SIZE:
            # Resize the image to a smaller size
            ratio = np.sqrt(MAX_IMAGE_SIZE / gray.size)
            resized = cv2.resize(gray, (0, 0), fx=ratio, fy=ratio)
        else:
            resized = gray


        # Apply image processing techniques to extract symbols
        # For example, you can use thresholding, contour detection, etc.
        # Here's a simple example using thresholding:
        _, threshold = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours in the thresholded image
        contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Iterate through the contours and extract symbols as individual images
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            symbol = gray[y:y+h, x:x+w]
            symbols.append(symbol)

    return symbols


In [ ]:

def preprocess_image(img):
    # Convert the image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image to obtain a binary image
    _, binary_img = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour
    contour_areas = [cv2.contourArea(contour) for contour in contours]
    largest_contour_index = np.argmax(contour_areas)
    largest_contour = contours[largest_contour_index]

    # Create a mask image for the largest contour
    mask = np.zeros_like(gray_img)
    cv2.drawContours(mask, [largest_contour], 0, 255, thickness=cv2.FILLED)

    # Apply the mask to the grayscale image
    masked_img = cv2.bitwise_and(gray_img, mask)

    # Resize the image to the desired input shape
    target_size = (32, 32)
    if masked_img.shape[0] != 0 and masked_img.shape[1] != 0:  # Check if the image is not empty
        resized_img = cv2.resize(masked_img, target_size, interpolation=cv2.INTER_AREA)
    else:
        resized_img = np.zeros(target_size, dtype=np.uint8)  # Return a black image if the image is empty

    # Normalize the pixel values to the range of [0, 1]
    normalized_img = resized_img / 255.0

    return normalized_img


print(pdf_path_ext)
for file in os.listdir(test_dir):
    print(file)
    if file.endswith('.pdf'):
        symbols = extract_symbols_from_pdf(pdf_path_ext)

        for symbol in symbols:
            try:
                # Convert symbol to grayscale
                symbol_gray = cv2.cvtColor(symbol, cv2.COLOR_BGR2GRAY)

                # Preprocess the symbol for prediction
                preprocessed_symbol = preprocess_image(symbol_gray)
                symbol_image = np.expand_dims(preprocessed_symbol, axis=-1)
                symbol_image = np.expand_dims(symbol_image, axis=0)

                # Perform prediction on the symbol image
                if symbol_image.size != 0:  # Check if symbol_image is not empty
                    prediction = model.predict(symbol_image)
                    predicted_label = np.argmax(prediction)

                    # Print the predicted symbol label
                    print("Symbol:", symbol)
                    print("Predicted Label:", predicted_label)
            except Exception as e:
                print("Error occurred during symbol preprocessing:", str(e))


## New trial 

In [5]:
# Get the list of symbol icon files
symbol_icon_files = os.listdir(train_dir)

# Set the input image dimensions
input_shape = (32, 32, 3)
input_shape = (64, 64, 3)

# Initialize lists to store images and labels
images = []
labels = []

In [6]:
# Initialize a dictionary to store symbol indices and filenames
symbol_index_to_filename = {}

# Load symbol icons and their labels
for i, symbol_icon_filename in enumerate(symbol_icon_files):
    label, ext = os.path.splitext(symbol_icon_filename)
    if ext.lower() in ['.jpeg', '.jpg', '.png']:
        # Add the filename to the dictionary with the corresponding symbol index
        symbol_index_to_filename[i] = symbol_icon_filename
        label = os.path.splitext(symbol_icon_filename)[0]  # Use the symbol_icon_filename as the label
        ext = os.path.splitext(symbol_icon_filename)[1]
        image_path = os.path.join(train_dir, symbol_icon_filename)
        image = load_img(image_path, target_size=input_shape[:2])
        image_array = img_to_array(image)
        images.append(image_array)
        labels.append(label)

# Convert the data into NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize the image data
images = images / 255.0

In [7]:
labels

array(['1-Pass Bubl Cap', '1-Pass Gen Tray', '1-Pass Sieve', ...,
       'Y Stop Check Valve', 'Y Strainer', '_CC_Line Number w Insu thk'],
      dtype='<U67')

In [8]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [9]:
# Create the CNN model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(np.unique(labels_encoded)), activation='softmax')  # Number of classes based on unique labels
])

In [10]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# Train the CNN model
model.fit(images, labels_encoded, epochs=10, batch_size=32)

Epoch 1/10
72/72 [==============================] - 3s 28ms/step - loss: 7.7745 - accuracy: 0.0000e+00
Epoch 2/10
72/72 [==============================] - 2s 28ms/step - loss: 7.7432 - accuracy: 0.0000e+00
Epoch 3/10
72/72 [==============================] - 2s 28ms/step - loss: 7.7431 - accuracy: 0.0000e+00
Epoch 4/10
72/72 [==============================] - 2s 28ms/step - loss: 7.7432 - accuracy: 0.0000e+00
Epoch 5/10
72/72 [==============================] - 2s 28ms/step - loss: 7.7431 - accuracy: 0.0000e+00
Epoch 6/10
72/72 [==============================] - 2s 28ms/step - loss: 7.7432 - accuracy: 0.0000e+00
Epoch 7/10
72/72 [==============================] - 2s 28ms/step - loss: 7.7432 - accuracy: 0.0000e+00
Epoch 8/10
72/72 [==============================] - 2s 28ms/step - loss: 7.7432 - accuracy: 0.0000e+00
Epoch 9/10
72/72 [==============================] - 2s 28ms/step - loss: 7.7432 - accuracy: 4.3516e-04
Epoch 10/10
72/72 [==============================] - 2s 29ms/step - loss:

#### Create a CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(set(labels)), activation='softmax'))

##### Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 30752)             0         
                                                                 
 dense (Dense)               (None, 64)                1968192   
                                                                 
 dense_1 (Dense)             (None, 2298)              149370    
                                                                 
Total params: 2,118,458
Trainable params: 2,118,458
Non-trainable params: 0
______________________________________________

In [26]:
# Helper function to convert PDF to images
def convert_pdf_to_images(sample_path):
    images = []
    # Convert each page of the PDF to an image
    pages = convert_from_path(sample_path, poppler_path=poppler_path)
    for page in pages:
        # Convert PIL image to numpy array
        image = np.array(page)
        
        # Resize the image while preserving aspect ratio
        max_dim = 1000
        scale = min(max_dim / image.shape[0], max_dim / image.shape[1])
        new_size = (int(image.shape[1] * scale), int(image.shape[0] * scale))
        resized_image = cv2.resize(image, new_size)
        
        images.append(resized_image)
    return images


# Define the input shape expected by the model
input_shape = (32, 32)
input_shape = (64,64)

# Define a function to preprocess the image
def preprocess_image(image):
    # Resize the image to the desired input shape
    resized_image = cv2.resize(image, input_shape[:2])
    
    # Normalize the image
    normalized_image = resized_image / 255.0
    
    # Add an extra dimension to match the expected input shape of the model
    preprocessed_image = np.expand_dims(normalized_image, axis=0)
    
    return preprocessed_image

In [27]:
# Loop over the PDF pages
pdf_path_ext = 'C:/Users/lsreeram/Downloads/_/Hex/AI PID/02 Test_Dataset/Test.pdf'
images = convert_pdf_to_images(pdf_path_ext)

In [28]:
for image in images:
    print(image.shape)

(772, 1000, 3)


### Make Predictions

In [29]:
import cv2
import numpy as np
from keras.models import load_model

# Define the input shape expected by the model
input_shape = (32, 32)
input_shape = (64,64)

# Define a function to preprocess the image
def preprocess_image(image):
    # Resize the image to the desired input shape
    resized_image = cv2.resize(image, input_shape[:2])
    
    # Normalize the image
    normalized_image = resized_image / 255.0
    
    # Add an extra dimension to match the expected input shape of the model
    preprocessed_image = np.expand_dims(normalized_image, axis=0)
    
    return preprocessed_image

# Define a function to extract symbols from an image
def extract_symbols(image):
    # Preprocess the image
    preprocessed_image = preprocess_image(image)
    
    # Make predictions using the symbol recognition model
    predictions = model.predict(preprocessed_image)
    
    # Post-processing (e.g., non-maximum suppression, thresholding)
    
    # Return the extracted symbols
    return predictions


# Loop over the PDF pages
pdf_path_ext = 'C:/Users/lsreeram/Downloads/_/Hex/AI PID/02 Test_Dataset/Test.pdf'
images = convert_pdf_to_images(pdf_path_ext)

for image in images:
    predictions = extract_symbols(image)

    # Process the extracted symbols (e.g., save, analyze, visualize)


1/1 [==============================] - 0s 16ms/step


In [30]:
# Interpret the predictions
predicted_symbols = []  # List to store the predicted symbols

for prediction in predictions:
    # Assuming prediction is a one-hot encoded vector
    predicted_label = np.argmax(prediction)
    predicted_symbols.append(predicted_label)

# Associate predicted symbols with their corresponding locations in the original drawing
# This step involves integrating the predicted symbols with the appropriate positions or connections in the drawing

# Print the predicted symbols
for symbol, predicted_symbol in zip(predictions, predicted_symbols):
    print("Symbol:", symbol)
    print("Predicted Symbol:", predicted_symbol)

Symbol: [0.00043507 0.00043572 0.00042438 ... 0.00044797 0.00043684 0.00043677]
Predicted Symbol: 2035


In [31]:
predicted_symbol_index = 1083
predicted_symbol_filename = symbol_index_to_filename[predicted_symbol_index]
print("Predicted Symbol Filename:", predicted_symbol_filename)

Predicted Symbol Filename: Alternate Oper P-T Break.Jpeg
